In [ ]:
%%python
import numpy as np
from timeit import timeit
import torch

class Matrix:
    def __init__(self, value, rows, cols):
        self.value = value
        self.rows = rows
        self.cols = cols
        
    def __getitem__(self, idxs):
        return self.value[idxs[0]][idxs[1]]
    
    def __setitem__(self, idxs, value):
        self.value[idxs[0]][idxs[1]] = value

Matplotlib is building the font cache; this may take a moment.


In [ ]:
%%python
def matmul_python(C, A, B):
    for m in range(C.rows):
        for k in range(A.cols):
            for n in range(C.cols):
                C[m, n] += A[m, k] * B[k, n]

def benchmark_matmul_python(M, N, K):
    A = Matrix(np.array(np.random.rand(M, K)), M, K)
    B = Matrix(np.array(np.random.rand(K, N)), K, N)
    C = Matrix(np.array(np.zeros((M, N))), M, N)
    secs = timeit(lambda: matmul_python(C, A, B), number=2)/2
    gflops = ((2*M*N*K)/secs) / 1e9
    print(gflops, "GFLOP/s", '\n', secs, 'seconds')
    return gflops

In [ ]:
benchmark_matmul_python(128,128,128).to_float64()

0.001945978976863244 GFLOP/s 
 2.1553696365008364 seconds


In [ ]:
%%python


def matmul_python(c,a,b):
    for i in range(c.cols):
        c[i,:]   = (a[i,:,None] * b[:,:]).sum(axis=0) # broadcast version
    return c

M,K,N = 128,128,128
A = Matrix(np.array(np.random.rand(M, K)), M, K)
B = Matrix(np.array(np.random.rand(K, N)), K, N)
C = Matrix(np.array(np.zeros((M, N))), M, N)
secs = timeit(lambda: matmul_python(C, A, B), number=2)/2
gflops = ((2*M*N*K)/secs) / 1e9
print(gflops, "GFLOP/s", '\n', secs, 'seconds')

# IF I didn't update this the next cell didnt use new matmul ?!
def benchmark_matmul_python(M, N, K):
    A = Matrix(np.array(np.random.rand(M, K)), M, K)
    B = Matrix(np.array(np.random.rand(K, N)), K, N)
    C = Matrix(np.array(np.zeros((M, N))), M, N)
    secs = timeit(lambda: matmul_python(C, A, B), number=2)/2
    gflops = ((2*M*N*K)/secs) / 1e9
    print(gflops, "GFLOP/s", '\n', secs, 'seconds')
    return gflops

2.2180149621294274 GFLOP/s 
 0.0018910170001618098 seconds


In [ ]:
# Using `list` insted of np.array is slower. 
python_gflops = benchmark_matmul_python(128,128,128).to_float64()

2.2364766680521035 GFLOP/s 
 0.0018754070006252732 seconds


In [ ]:
%%python
import torch

def matmul_python(A,B): return torch.einsum('ik,kj->ij', A[:,:], B[:,:])

# This one need tensors
M,K,N = 128,128,128
A = Matrix(torch.tensor(np.random.rand(M, K)), M, K)
B = Matrix(torch.tensor(np.random.rand(K, N)), K, N)
C = Matrix(torch.tensor(np.zeros((M, N))), M, N)

secs = timeit(lambda: matmul_python(A, B), number=2)/2
gflops = ((2*M*N*K)/secs) / 1e9
print(gflops, "GFLOP/s", '\n', secs, 'seconds')

0.023808392861640604 GFLOP/s 
 0.17616913600068074 seconds


# Now let's use Mojo

for mojo test, we will assume pythons
2 GFLOP/s is the best python can do.

First, let's import modules that we are going to use throughout the notebook.

In [ ]:
from benchmark import Benchmark
from sys.intrinsics import strided_load
from utils.list import VariadicList
from math import div_ceil, min
from memory import memset_zero
from memory.unsafe import DTypePointer
from random import rand, random_float64
from sys.info import simdwidthof
from runtime.llcl import Runtime

In [ ]:
# This exactly the same Python implementation, 
# but is infact Mojo code!
def matmul_untyped(C, A, B):
    for m in range(C.rows):
        for k in range(A.cols):
            for n in range(C.cols):
                C[m, n] += A[m, k] * B[k, n]

We need to change our benchmark to work with mojo as well

In [ ]:
fn matrix_getitem(self: object, i: object) raises -> object:
    return self.value[i]


fn matrix_setitem(self: object, i: object, value: object) raises -> object:
    self.value[i] = value
    return None


fn matrix_append(self: object, value: object) raises -> object:
    self.value.append(value)
    return None


fn matrix_init(rows: Int, cols: Int) raises -> object:
    let value = object([])
    return object(
        Attr("value", value), Attr("__getitem__", matrix_getitem), Attr("__setitem__", matrix_setitem), 
        Attr("rows", rows), Attr("cols", cols), Attr("append", matrix_append),
    )

def benchmark_matmul_untyped(M: Int, N: Int, K: Int, python_gflops: Float64):
    C = matrix_init(M, N)
    A = matrix_init(M, K)
    B = matrix_init(K, N)
    for i in range(M):
        c_row = object([])
        b_row = object([])
        a_row = object([])
        for j in range(N):
            c_row.append(0.0)
            b_row.append(random_float64(-5, 5))
            a_row.append(random_float64(-5, 5))
        C.append(c_row)
        B.append(b_row)
        A.append(a_row)

    @parameter
    fn test_fn():
        try:
            _ = matmul_untyped(C, A, B)
        except:
            pass

    let secs = Float64(Benchmark().run[test_fn]()) / 1_000_000_000
    _ = (A, B, C)
    let gflops = ((2*M*N*K)/secs) / 1e9
    let speedup : Float64 = gflops / python_gflops
    print(gflops, "GFLOP/s,o took:",secs,"s a", speedup.value, "x speedup over Python")

In [ ]:
benchmark_matmul_untyped(128, 128, 128, python_gflops)

0.023230609161565729 GFLOP/s,o took: 0.18055075400000001 s a 0.010387145769689077 x speedup over Python


We need to keep I'm mind that we use np.arrays in Python and I believe it is run in C. \
Let's try improving it.

## Adding types to the Python implementation

We need to define a `Matrix` struct, and we will set data type to `Float32` for conciseness

In [ ]:
struct Matrix:
    var data: DTypePointer[DType.float32]
    var rows: Int
    var cols: Int

    fn __init__(inout self, rows: Int, cols: Int):
        self.data = DTypePointer[DType.float32].alloc(rows * cols)
        rand(self.data, rows*cols)
        self.rows = rows
        self.cols = cols

    fn __del__(owned self):
        self.data.free()

    fn zero(inout self):
        memset_zero(self.data, self.rows * self.cols)

    @always_inline
    fn __getitem__(self, y: Int, x: Int) -> Float32:
        return self.load[1](y, x)

    @always_inline
    fn load[nelts:Int](self, y: Int, x: Int) -> SIMD[DType.float32, nelts]:
        return self.data.simd_load[nelts](y * self.cols + x)

    @always_inline
    fn __setitem__(self, y: Int, x: Int, val: Float32):
        return self.store[1](y, x, val)

    @always_inline
    fn store[nelts:Int](self, y: Int, x: Int, val: SIMD[DType.float32, nelts]):
        self.data.simd_store[nelts](y * self.cols + x, val)

With the above `Matrix` type, we can effectively copy and paste the Python implementation and just add type annotations:

In [ ]:
# Note that C, A, and B have types.
fn matmul_naive(C: Matrix, A: Matrix, B: Matrix):
    for m in range(C.rows):
        for k in range(A.cols):
            for n in range(C.cols):
                C[m, n] += A[m, k] * B[k, n]

We are going to benchmark the implementations as we improve, so let’s write a helper function that will do that for us:

In [ ]:
@always_inline
fn benchmark[
    func: fn (Matrix, Matrix, Matrix) -> None](M: Int, N: Int, K: Int, base_gflops: Float64):
    var C = Matrix(M, N)
    C.zero()
    var A = Matrix(M, K)
    var B = Matrix(K, N)

    @always_inline
    @parameter
    fn test_fn():
        _ = func(C, A, B)

    let secs = Float64(Benchmark().run[test_fn]()) / 1_000_000_000
    # Prevent the matrices from being freed before the benchmark run
    _ = (A, B, C)
    let gflops = ((2 * M * N * K) / secs) / 1e9
    let speedup: Float64 = gflops / base_gflops
    # print(gflops, "GFLOP/s", speedup, " speedup")
    print(gflops, "GFLOP/s,o took:",secs,"s a", speedup.value, "x speedup over Python")

In [ ]:
print("for MNK size 128")
benchmark[matmul_naive](128, 128, 128, python_gflops)
print("for MNK size 512")
benchmark[matmul_naive](512, 512, 512, python_gflops)

for MNK size 128
5.2443680894453806 GFLOP/s,o took: 0.00079977300000000004 s a 2.3449241230014932 x speedup over Python
for MNK size 512
3.4627032018395414 GFLOP/s,o took: 0.077521936 s a 1.5482849659484443 x speedup over Python


Adding type annotations gives a huge improvement compared to the untyped version.

# Vectorizing the inner most loop

We can do better than that, by utilizing the vector instructions.\
Rather than assuming a vector width, we query the `SIMD` width of the specified dtype using `sidm_width`. This makes our code portable as we transition to other hardware.

In [ ]:
# Mojo has SIMD vector types, we can vectorize the Matmul code as follows.

alias nelts = simdwidthof[DType.float32]() # The SIMD vector width. ## I assume its width of a float32.
fn matmul_vectorized(C: Matrix, A: Matrix, B: Matrix):
    for m in range(C.rows):
        for k in range(A.cols):
            for nv in range(0, C.cols,nelts):
                C.store[nelts](m, nv, C.load[nelts](m,nv) + A[m,k] * B.load[nelts](k,nv))

            # Handle remaining elements with scalars.
            for n in range(nelts * (C.cols//nelts), C.cols):
                C[m,n] += A[m,k] + B[k,n]

In [ ]:
print("for MNK size 128")
benchmark[matmul_vectorized](128, 128, 128, python_gflops)
print("for MNK size 512")
benchmark[matmul_vectorized](512, 512, 512, python_gflops)
print("for MNK size 1024")
benchmark[matmul_vectorized](1024, 1024, 1024, python_gflops)

for MNK size 128
32.704379761245697 GFLOP/s,o took: 0.00012824900000000001 s a 14.623170555912894 x speedup over Python
for MNK size 512
31.009379791509105 GFLOP/s,o took: 0.0086565889999999993 s a 13.865282045851718 x speedup over Python
for MNK size 1024
20.674823960115106 GFLOP/s,o took: 0.103869501 s a 9.2443727473008632 x speedup over Python


Vectorization is a common optimization, and Mojo provides a higher-order function that performs vectorization for you. \
The `vectorize` function takes a vector width and a function which is parametric on the vector width and is going to be evaluated in a vectorized manner.

In [ ]:
# Simplify the code by using the builin vetorize function
from algorithm import vectorize
fn matmul_vectorized_1(C: Matrix, A: Matrix, B: Matrix):
    for m in range(C.rows):
        for k in range(A.cols):
            @parameter
            fn dot[nelts: Int](n: Int):
                C.store[nelts](m, n, C.load[nelts](m,n) + A[m,k] * B.load[nelts](k,n))
            
            vectorize[nelts, dot](C.cols)

There is only a slight difference in terms of performance between the two implementations:

In [ ]:
print("for MNK size 128")
benchmark[matmul_vectorized_1](128, 128, 128, python_gflops)
print("for MNK size 512")
benchmark[matmul_vectorized_1](512, 512, 512, python_gflops)
print("for MNK size 1024")
benchmark[matmul_vectorized_1](1024, 1024, 1024, python_gflops)

for MNK size 128
33.653507927338083 GFLOP/s,o took: 0.00012463200000000001 s a 15.047556009895317 x speedup over Python
for MNK size 512
31.519860466716477 GFLOP/s,o took: 0.0085163909999999999 s a 14.093534225943534 x speedup over Python
for MNK size 1024
24.845451415150997 GFLOP/s,o took: 0.086433674000000002 s a 11.109193210046119 x speedup over Python


In [ ]:

@always_inline
fn benchmark_parallel[
    func: fn (Matrix, Matrix, Matrix, Runtime) -> None
](M: Int, N: Int, K: Int, base_gflops: Float64):
    var C = Matrix(M, N)
    C.zero()
    var A = Matrix(M, K)
    var B = Matrix(K, N)

    with Runtime() as rt:
        @always_inline
        @parameter
        fn test_fn():
            _ = func(C, A, B, rt)

        let secs = Float64(Benchmark().run[test_fn]()) / 1_000_000_000
        # Prevent the matrices from being freed before the benchmark run
        _ = (A, B, C)
        let gflops = ((2 * M * N * K) / secs) / 1e9
        let speedup: Float64 = gflops / base_gflops
        # print(gflops, "GFLOP/s", speedup, " speedup")
        print(gflops, "GFLOP/s,o took:",secs,"s a", speedup.value, "x speedup over Python")

In [ ]:
# Parallelize the code by using the builtin parallelize function
from algorithm import parallelize
fn matmul_parallelized(C: Matrix, A: Matrix, B: Matrix, rt: Runtime):
    @parameter
    fn calc_row(m: Int):
        for k in range(A.cols):
            @parameter
            fn dot[nelts : Int](n : Int):
                C.store[nelts](m,n, C.load[nelts](m,n) + A[m,k] * B.load[nelts](k,n))
            vectorize[nelts, dot](C.cols)
        
    parallelize[calc_row](rt, C.rows)

In [ ]:
print("for MNK size 128")
benchmark_parallel[matmul_parallelized](128, 128, 128, python_gflops)
print("for MNK size 512")
benchmark_parallel[matmul_parallelized](512, 512, 512, python_gflops)
print("for MNK size 1024")
benchmark_parallel[matmul_parallelized](1024, 1024, 1024, python_gflops)

for MNK size 128
33.430338583179243 GFLOP/s,o took: 0.00012546399999999999 s a 14.947769883195763 x speedup over Python
for MNK size 512
32.611657516401166 GFLOP/s,o took: 0.0082312730000000008 s a 14.58171148496927 x speedup over Python
for MNK size 1024
33.629564651126195 GFLOP/s,o took: 0.063857016000000003 s a 15.036850207973071 x speedup over Python


In [ ]:
from algorithm import Static2DTileUnitFunc as Tile2DFunc
# Perform 2D tiling on the iteration space defined by end_x and end_y.
fn tile[tiled_fn: Tile2DFunc, tile_x: Int, tile_y: Int](end_x: Int, end_y: Int):
    # Note: this assumes that ends are multiples of the tiles.
    for y in range(0, end_y, tile_y):
        for x in range(0, end_x, tile_x):
            tiled_fn[tile_x, tile_y](x, y)

In [ ]:
# Use the above tile function to perform tiled matmul.
fn matmul_tiled_parallelized(C: Matrix, A: Matrix, B: Matrix, rt: Runtime):
    @parameter
    fn calc_row(m: Int):
        @parameter
        fn calc_tile[tile_x: Int, tile_y: Int](x: Int, y: Int):
            for k in range(y, y + tile_y):
                @parameter
                fn dot[nelts : Int,](n : Int):
                    C.store[nelts](m,n + x, C.load[nelts](m,n+x) + A[m,k] * B.load[nelts](k,n+x))
                vectorize[nelts, dot](tile_x)

        # We hardcode the tile factor to be 4.
        alias tile_size = 4
        tile[calc_tile, nelts * tile_size, tile_size](A.cols, C.cols)

    parallelize[calc_row](rt, C.rows)

In [ ]:
print("for MNK size 128")
benchmark_parallel[matmul_tiled_parallelized](128, 128, 128, python_gflops)
print("for MNK size 512")
benchmark_parallel[matmul_tiled_parallelized](512, 512, 512, python_gflops)
print("for MNK size 1024")
benchmark_parallel[matmul_tiled_parallelized](1024, 1024, 1024, python_gflops)

for MNK size 128
28.202120721072063 GFLOP/s,o took: 0.00014872299999999999 s a 12.610067041582495 x speedup over Python
for MNK size 512
33.875645101306226 GFLOP/s,o took: 0.0079241429999999998 s a 15.146880620404943 x speedup over Python
for MNK size 1024
40.149339254628522 GFLOP/s,o took: 0.053487396999999999 s a 17.952049233581882 x speedup over Python


In [ ]:
# Unroll the vectorized loop by a constant factor.
from algorithm import vectorize_unroll
fn matmul_tiled_unrolled_parallelized(C: Matrix, A: Matrix, B: Matrix, rt: Runtime):
    @parameter
    fn calc_row(m: Int):
        @parameter
        fn calc_tile[tile_x: Int, tile_y: Int](x: Int, y: Int):
            for k in range(y, y + tile_y):
                @parameter
                fn dot[nelts : Int,](n : Int):
                    C.store[nelts](m,n+x, C.load[nelts](m,n+x) + A[m,k] * B.load[nelts](k,n+x))

                # Vectorize by nelts and unroll by tile_x/nelts
                # Here unroll factor is 4
                vectorize_unroll[nelts, tile_x//nelts, dot](tile_x)

        alias tile_size = 4
        tile[calc_tile, nelts*tile_size, tile_size](A.cols, C.cols)
      
    parallelize[calc_row](rt, C.rows)

In [ ]:
print("for MNK size 128")
benchmark_parallel[matmul_tiled_unrolled_parallelized](128, 128, 128, python_gflops)
print("for MNK size 512")
benchmark_parallel[matmul_tiled_unrolled_parallelized](512, 512, 512, python_gflops)
print("for MNK size 1024")
benchmark_parallel[matmul_tiled_unrolled_parallelized](1024, 1024, 1024, python_gflops)

for MNK size 128
31.139501388332068 GFLOP/s,o took: 0.00013469400000000001 s a 13.923463559069244 x speedup over Python
for MNK size 512
35.392660421030008 GFLOP/s,o took: 0.0075844950000000001 s a 15.825186520660568 x speedup over Python
for MNK size 1024
37.444583085873177 GFLOP/s,o took: 0.057350983000000001 s a 16.742666543660462 x speedup over Python


Maybe it's not `70_000`x times faster, but it is still a lot faster. Keep in minds that we used NumPy in python, and it is run in C(I believe). \
It's still much useful for debugging and testing purposes, also its version `0.3` so we might get better performance in the future.